# Trabalho Final
Neste trabalho, vamos buscar o reconhecimento dos dígitos da Linguagem de Sinais. Para tal, vamos nos utilizar de Modelos conhecidos de Deep Learning e também nos aventurar na criação de próprios.

------

## Imports

In [1]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

import inf619utils

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Dataset
O dataset é composto por 10 classes (dígitos de 0 a 9) com aproximadamente 205 imagens por classe. 
O conjunto foi dividido em 60% para treinamento, 15% para validação e 20% para teste.
As imagens estão divididas em blocos por classe.

** IMPORTANTE NÃO ALTERAR O NOME/LOCAL DAS IMAGENS** 

In [2]:
datasetDir = "./Dataset"
nbClasses = 10
input_shape = (100,100,3)

train_files_adam = {}
val_files_adam = {}
test_files_adam = {}

train_files_adam, val_files_adam, test_files_adam = inf619utils.splitData(datasetDir, nbClasses)

train_files_adadelta = {}
val_files_adadelta = {}
test_files_adadelta = {}

train_files_adadelta, val_files_adadelta, test_files_adadelta = inf619utils.splitData(datasetDir, nbClasses)

train_files_sgd = {}
val_files_sgd = {}
test_files_sgd = {}

train_files_sgd, val_files_sgd, test_files_sgd = inf619utils.splitData(datasetDir, nbClasses)

In [3]:
#plot the images from imgList
def plotImagesFromBatch(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])


# Se quiser visualizar algum bloco de imagens, descomentar as linhas abaixo
# inf619utils.plotImages(val_files)
# inf619utils.plotImages(train_files)
# inf619utils.plotImages(test_files)

In [4]:
trainSetSize = inf619utils.getDatasetSize(train_files_adam)
valSetSize = inf619utils.getDatasetSize(val_files_adam)
testSetSize = inf619utils.getDatasetSize(test_files_adam)

print("# images in Train set: ", trainSetSize)
print("# images in Val set: ", valSetSize)
print("# images in Test set: ", testSetSize)

# images in Train set:  1242
# images in Val set:  309
# images in Test set:  511


## Definição do modelo 

In [5]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(input_shape):
    img_input = Input(shape=input_shape) #placeholder
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

In [6]:
#for batch, labels in inf619utils.loadDatasetInBatches(train_files_adam, batch_size=32, input_shape=input_shape, nbClasses=nbClasses, shouldAugmentData=True):
#    print(batch.shape, labels.shape)
    #plotImagesFromBatch(batch)
#    break


# Modificação do modelo 

In [7]:
# Definir o modelo base da squeezeNet 
squeezeNetModel = SqueezeNet(input_shape)

# Escolher a camada que será o ponto de partida 
x = squeezeNetModel.get_layer(name="fire9/concat").output

#print([layer.name for layer in squeezeNetModel.layers])
#print("\n\nFreeze layers up until ", squeezeNetModel.layers[-20].name)

for layer in squeezeNetModel.layers:
    layer.trainable = True#        layer.trainable = False

x = Convolution2D(1024, (1, 1), padding='valid', name='conv10_new')(x)
x = Activation('relu', name='relu_conv10_new')(x)
x = Dropout(0.5)(x)
x = Convolution2D(nbClasses, (1, 1), padding='valid', name='conv11_new')(x)
x = Activation('relu', name='relu_conv11_new')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss_new')(x)


# Não se esqueça de definir o nome modelo, onde baseSqueezeNetModel 
# é o modelo base da Squeeze que vc definiu ali em cima
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adam')
model2 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adadelta')
model3 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_sgd')

# Treinamento do Modelo

In [8]:
from keras.optimizers import SGD, Adam
#Compile o modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model3.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True), metrics=['accuracy'])

import keras.callbacks as callbacks

tbCallBack = callbacks.TensorBoard(log_dir = "./logs_squeeze8")
tbEarly = callbacks.EarlyStopping(monitor='val_acc',min_delta=0,patience=10,verbose=0, mode='auto')
tbModelAdamChk = callbacks.ModelCheckpoint('.modeladam_weights.hdf5', save_best_only=True, monitor='val_acc', mode='max')
tbModelAdadeltaChk = callbacks.ModelCheckpoint('.modeladadelta_weights.hdf5', save_best_only=True, monitor='val_acc', mode='max')
tbModelSgdChk = callbacks.ModelCheckpoint('.modelsgd_weights.hdf5', save_best_only=True, monitor='val_acc', mode='max')


In [ ]:
#Definir tamanho do batch e número de épocas
batch_size = 8
epochs = 30

#Criação dos generators
trainGenerator_adam = inf619utils.loadDatasetInBatches(train_files_adam, batch_size = batch_size, input_shape=input_shape, nbClasses=nbClasses,  shouldAugmentData=True)
valGenerator_adam = inf619utils.loadDatasetInBatches(val_files_adam, batch_size = batch_size)

#Fit nos dados
hist = model.fit_generator(trainGenerator_adam, 
                    steps_per_epoch= int(trainSetSize / batch_size), 
                    epochs = epochs,
                    validation_data = valGenerator_adam,  
                    validation_steps = int(valSetSize / batch_size),
                    callbacks=[tbCallBack, tbEarly, tbModelAdamChk])

In [ ]:
#Definir tamanho do batch e número de épocas
batch_size = 8
epochs = 30

#Criação dos generators
trainGenerator_adadelta = inf619utils.loadDatasetInBatches(train_files_adadelta, batch_size = batch_size, input_shape=input_shape, nbClasses=nbClasses, shouldAugmentData=True)
valGenerator_adadelta = inf619utils.loadDatasetInBatches(val_files_adadelta, batch_size = batch_size)

#Fit nos dados
hist = model2.fit_generator(trainGenerator_adadelta, 
                    steps_per_epoch= int(trainSetSize / batch_size), 
                    epochs = epochs,
                    validation_data = valGenerator_adadelta,  
                    validation_steps = int(valSetSize / batch_size),
                    callbacks=[tbCallBack, tbEarly, tbModelAdadeltaChk])

In [ ]:
#Definir tamanho do batch e número de épocas
batch_size = 8
epochs = 30

#Criação dos generators
trainGenerator_sgd = inf619utils.loadDatasetInBatches(train_files_sgd, batch_size = batch_size, input_shape=input_shape, nbClasses=nbClasses, shouldAugmentData=True)
valGenerator_sgd = inf619utils.loadDatasetInBatches(val_files_sgd, batch_size = batch_size)

#Fit nos dados
hist = model3.fit_generator(trainGenerator_sgd, 
                    steps_per_epoch= int(trainSetSize / batch_size), 
                    epochs = epochs,
                    validation_data = valGenerator_sgd,  
                    validation_steps = int(valSetSize / batch_size),
                    callbacks=[tbCallBack, tbEarly, tbModelSgdChk])

Epoch 1/30
155/155 [==============================] - 8s 49ms/step - loss: 2.2775 - acc: 0.1460 - val_loss: 2.1626 - val_acc: 0.2763
Epoch 2/30
155/155 [==============================] - 9s 59ms/step - loss: 2.0322 - acc: 0.2847 - val_loss: 1.7374 - val_acc: 0.3987
Epoch 3/30
155/155 [==============================] - 16s 101ms/step - loss: 1.5950 - acc: 0.4653 - val_loss: 1.2459 - val_acc: 0.5681
Epoch 4/30
155/155 [==============================] - 17s 112ms/step - loss: 1.0955 - acc: 0.6169 - val_loss: 0.8502 - val_acc: 0.7542
Epoch 5/30
155/155 [==============================] - 17s 113ms/step - loss: 0.7617 - acc: 0.7379 - val_loss: 0.6854 - val_acc: 0.7508
Epoch 6/30
155/155 [==============================] - 17s 112ms/step - loss: 0.5794 - acc: 0.8064 - val_loss: 0.5693 - val_acc: 0.7973
Epoch 7/30
155/155 [==============================] - 17s 112ms/step - loss: 0.4640 - acc: 0.8379 - val_loss: 0.4861 - val_acc: 0.8306
Epoch 8/30
155/155 [==============================] - 17s 1

# Aplicação do Modelo no Conjunto de Testes

In [ ]:
#Criação do generator p/ o conjunto de teste
testGenerator_adam = inf619utils.loadDatasetInBatches(test_files_adam, batch_size=batch_size)

# Load Best weights saved
model.load_weights(filepath='.modeladam_weights.hdf5')

#Teste
metrics = model.evaluate_generator(testGenerator_adam, 
                                   steps=int(testSetSize/batch_size), 
                                   verbose=1)

print("Test Loss Adam---> ", metrics[0])
print("Test Accuracy Adam---> ", metrics[1])    #Test is balanced, so Acc is normalized

In [ ]:
#Criação do generator p/ o conjunto de teste
testGenerator_adadelta = inf619utils.loadDatasetInBatches(test_files_adadelta, batch_size=batch_size)

# Load Best weights saved
model2.load_weights(filepath='.modeladadelta_weights.hdf5')

#Teste
metrics = model2.evaluate_generator(testGenerator_adadelta, 
                                   steps=int(testSetSize/batch_size), 
                                   verbose=1)

print("Test Loss Adadelta---> ", metrics[0])
print("Test Accuracy Adadelta---> ", metrics[1])    #Test is balanced, so Acc is normalized

In [ ]:
#Criação do generator p/ o conjunto de teste
testGenerator_sgd = inf619utils.loadDatasetInBatches(test_files_sgd, batch_size=batch_size)

# Load Best weights saved
model3.load_weights(filepath='.modelsgd_weights.hdf5')

#Teste
metrics = model3.evaluate_generator(testGenerator_sgd, 
                                   steps=int(testSetSize/batch_size), 
                                   verbose=1)

print("Test Loss SGD---> ", metrics[0])
print("Test Accuracy SGD---> ", metrics[1])    #Test is balanced, so Acc is normalized

In [ ]:
model.save('squeezenet_adam_96.h5')
model2.save('squeezenet_adadelta_984.h5')
model3.save('squeezenet_sgd_986.h5')